## Averaged flows


In [1]:
import cv2
import glob
import matplotlib.pyplot as plt
import numpy as np
from natsort import natsorted
from scipy.ndimage import gaussian_filter
import os
import sys
sys.path.append('../vasco_scripts')  # add the relative path to the folder
from defects import *  # import the module from the folder
%matplotlib qt

def divergence_npgrad(flow):
    # flow = np.swapaxes(flow, 0, 1)
    Fx, Fy = flow[:, :, 0], flow[:, :, 1]
    dFx_dx = np.gradient(Fx, axis=0)
    dFy_dy = np.gradient(Fy, axis=1)
    return dFx_dx + dFy_dy


def analyze_defects(img):
    # Calculate mgrid
    yy, xx = np.mgrid[0:img.shape[0], 0:img.shape[1]]
    
    # Calculate orientation analysis
    ori, coh, E = orientation_analysis(img, 25)
    
    # Compute topological charges
    k = compute_topological_charges(-ori, int_area='cell', origin='lower')
    
    # Localize defects
    defects = localize_defects(k, x_grid=xx, y_grid=yy)
    
    # Compute defect orientation
    compute_defect_orientations(-ori, defects, method='interpolation', x_grid=xx[0,:], y_grid=yy[:,0], interpolation_radius=5, min_sep=1)
    
    # Filter defects by charge
    plushalf = defects[defects['charge']==.5]
    minushalf = defects[defects['charge']==-.5]
    
    return ori, plushalf, minushalf



# Defect dynamics time lapse

In [9]:

# image_list = glob.glob(r"C:/Users/victo/Downloads/SB_lab/RPE1_C2C12_/Test001/*.tif")
image_list_all = glob.glob(r"C:\Users\victo\My Drive\Curie\Trinish\30_01_18_RPE1_C2C12_5x_phase_nuclei\*s14*.tif")
image_list_all = natsorted(image_list_all, key=lambda y: y.lower())
image_list = image_list_all[50:250:1]

fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.set_axis_off()
extent = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())

u = np.zeros_like(cv2.imread(image_list[0])[:,:,0], dtype=np.float32)
v = np.zeros_like(u)


for im1, im2 in zip(image_list[:-1], image_list[1:]):
    ax.clear()
    img1 = cv2.imread(im1)[:,:,0]
    img2 = cv2.imread(im2)[:,:,0]
    flow = cv2.calcOpticalFlowFarneback(img1,img2, None, 0.5, 3, 
        winsize=15, iterations=3, poly_n=5, poly_sigma=1.2, flags=0)  
    u += flow[..., 0]
    v += flow[..., 1]

    flow = np.stack((u, v), axis=-1)
    divergence = gaussian_filter(divergence_npgrad(flow), sigma=30)

    # Apply Contrast Limited Adaptive Histogram Equalization
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_clahe = clahe.apply(img1)
    ax.imshow(255-img_clahe, "gray")
    ax.imshow(divergence, "jet", alpha=.3, vmin=-divergence.max()/2, vmax=divergence.max()/2)

    ori, plushalf, minushalf = analyze_defects(img1)

    alpha_half, scale_half = .9, 40
    ax.plot(plushalf['x'], plushalf['y'],'ro',markersize=6, alpha=1)
    ax.quiver(plushalf['x'], plushalf['y'], 
        np.cos(plushalf['ang1']), -np.sin(plushalf['ang1']), 
        headaxislength=0, headwidth=0, headlength=0, color='r', scale=scale_half, alpha=alpha_half)

    ax.plot(minushalf['x'], minushalf['y'],'bo',markersize=6, alpha=1)
    for i in range(3):
        ax.quiver(minushalf['x'], minushalf['y'], 
            np.cos(minushalf['ang'+str(i+1)]), -np.sin(minushalf['ang'+str(i+1)]), 
            headaxislength=0, headwidth=0, headlength=0, color='b', scale=scale_half, alpha=alpha_half)
    
    
    save_path = os.path.join(
        r"C:\Users\victo\Downloads\SB_lab\RPE1_C2C12_", 
        'Nematic', 
        os.path.splitext(os.path.basename(im1))[0]
        )
    # break
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    fig.savefig(save_path, bbox_inches=extent.expanded(1.15, 1.15))

    # break



## Add Orientation

In [55]:
from scipy.ndimage import gaussian_filter
import sys
sys.path.append('../vasco_scripts')  # add the relative path to the folder
from defects import *  # import the module from the folder

def shear(flow):
    # flow = np.swapaxes(flow, 0, 1)
    Fx, Fy = flow[:, :, 0], flow[:, :, 1]
    dFx_dx, dFx_dy = np.gradient(Fx)
    dFy_dx, dFy_dy = np.gradient(Fy)
    return dFx_dy + dFy_dx

def divergence_npgrad(flow):
    # flow = np.swapaxes(flow, 0, 1)
    Fx, Fy = flow[:, :, 0], flow[:, :, 1]
    dFx_dx = np.gradient(Fx, axis=0)
    dFy_dy = np.gradient(Fy, axis=1)
    return dFx_dx + dFy_dy

flow = np.stack((u, v), axis=-1)
divergence = gaussian_filter(divergence_npgrad(flow), sigma=30)

img = cv2.imread(image_list[-1])[..., 0]

yy, xx = np.mgrid[0:img.shape[0], 0:img.shape[1]]
ori, coh, E = orientation_analysis(img, 25)
ori, coh, E = orientation_analysis(img, 25)
k = compute_topological_charges(-ori, int_area='cell', origin='lower')
defects = localize_defects(k, x_grid=xx, y_grid=yy)
compute_defect_orientations(-ori, defects, method='interpolation', x_grid=xx[0,:], y_grid=yy[:,0], interpolation_radius=5,  min_sep=1)
plushalf = defects[defects['charge']==.5]
minushalf = defects[defects['charge']==-.5]
